In [153]:
from flask import Flask, render_template_string, request, url_for, jsonify
import os

app = Flask(__name__)

# Initialize the neutrality score at 0.5
neutrality_score = 0.5
# create lists to store liked and disliked posts
liked_posts = set()
disliked_posts = set()
# List of available images in the static folder
folder_path = "static"

# Get a list of all file names in the folder
available_images = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

#start page template
start_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Start Page</title>
    <link href="https://fonts.googleapis.com/css2?family=Baskervville&display=swap" rel="stylesheet">
    <style>
    body {
        display: grid;
        place-items: center;
        font-family: 'Baskervville', serif;
        background-color: #F5EFE0;
    
    }
    .start-button {
         margin-top: 30px;
            padding: 15px 30px;
            font-family: inherit;
            font-size: 30px;
            background-color: #E8DEC9;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            }
    @keyframes shrink {
        0% {transform: scale(1); }
        100% {transform: scale(0.8); }
        }
    .shrink-image {
        width: 600px;
        height: auto;
        animation: shrink 2s ease-in-out forwards;
        animation-delay: 0.3s;
    }
    </style>
</head>

<body>
    
    <img src="{{ url_for('static', filename='rabbitlogo-removebg.png') }}" alt="Logo" class="shrink-image" >
    <h2 style="text-align:center;"> Welcome to Rabbit, an interactive environment <br> where you can experience World War 1 brought into the modern day. <br>
    Click start to be brought into the social media simulation. </h2>
     <form action="/home" method="get">
        <button type="submit" class="start-button">Start</button>
    </form>
</body>
</html>

"""

# Main page HTML template with clickable images and image scores
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Flask Button Example</title>
     <style>
        body {
            background-color: #F5EFE0;
            padding-top: 10px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

     
        .banner img {
            height: 120%;
            width: auto;
            object-fit: contain;
        }
        

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 450px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
    .image-button {
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    .image-button img {
        width: 5px;
        height: auto;
        }
}
        
<button class="like-btn" data-post-id="1">Like</button>
<button class="dislike-btn" data-post-id="1">Dislike</button>
<p>Neutrality Score: <span id="neutrality-score">0.5</span></p>
document.querySelectorAll(".like-btn").forEach(button => {
    button.addEventListener("click", function () {
        const postId = this.getAttribute("data-post-id");

        fetch("/like", {
            method: "POST",
            headers: { "Content-Type": "application/x-www-form-urlencoded" },
            body: new URLSearchParams({ post_id: postId })
        })
        .then(response => response.json())
        .then(data => {
            if (data.success) {
                document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);
            } else {
                alert(data.message);
            }
        })
        .catch(error => console.error("Error:", error));
    });
});

document.querySelectorAll(".dislike-btn").forEach(button => {
    button.addEventListener("click", function () {
        const postId = this.getAttribute("data-post-id");

        fetch("/dislike", {
            method: "POST",
            headers: { "Content-Type": "application/x-www-form-urlencoded" },
            body: new URLSearchParams({ post_id: postId })
        })
        .then(response => response.json())
        .then(data => {
            if (data.success) {
                document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);
            } else {
                alert(data.message);
            }
        })
        .catch(error => console.error("Error:", error));
    });
});
    </style>
</head>

<body>
        
    <div class="banner">
        <img src="{{ url_for('static', filename='rabbitlogo-removebg.png') }}" alt="Logo" >
    </div>


    <div class="image-container">

        <form method="POST" action="/button-clicked/{{ images[0].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
                <img src="{{ url_for('static', filename=images[0].filename) }}" alt="Click to see {{ images[0].filename }}" >
            </button>
        
        </form>
        
        <form method="POST" action="/button-clicked/{{ images[1].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
                <img src="{{ url_for('static', filename=images[1].filename) }}" alt="Click to see {{ images[1].filename }}" >
            </button>
    
        </form>

    </div>

</body>

</html>
"""

# Template to display the clicked image
button_clicked_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Button Clicked</title>
</head>
<body>
    <p>Current Neutrality Score: {{ neutrality_score }}</p>
    <img src="{{ url_for('static', filename=image_filename) }}" alt="Clicked Image" style="width:300px;height:auto;">
    
    <p>Image Score: {{ image_score }}</p>  <!-- Display the image score here -->
    
    <form action="/home" method="get">
        <button type="submit">Back to Home</button>
    </form>
</body>
</html>
"""


@app.route("/", methods=["GET"])
def start():
        return render_template_string(start_template)

@app.route("/home", methods=["GET"])
def home():
    global neutrality_score
    # Randomly select two different images for display
    nr_images = len(available_images)
    
    # Calculate image score based on position
    images = []
    for idx, filename in enumerate(available_images):
        image_score = idx / nr_images  # Image score calculation
        images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Select two images based on the neutrality_score
    
    if neutrality_score == 1:
        image1 = int(neutrality_score * nr_images)-1
        image2 = image1 - 1
    elif neutrality_score == 0:
        image1 = int(neutrality_score * nr_images)
        image2 = image1 + 1
    else:
        image1 = int(neutrality_score * nr_images)
        image2 = image1 + 1

    selected_images = [images[image1], images[image2]]
    return render_template_string(html_template, neutrality_score=neutrality_score, images=selected_images)

@app.route("/button-clicked/<image_filename>", methods=["POST"])
def button_clicked(image_filename):
    global neutrality_score
    
    # Recalculate the image list (just like in the home route) to get the image score
    nr_images = len(available_images)
    images = []
    for idx, filename in enumerate(available_images):
        image_score = idx / nr_images  # Image score calculation
        images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Find the image with the corresponding filename
    image = next(image for image in images if image['filename'] == image_filename)
    image_score = image['score']  # Get the image score
    
    return render_template_string(button_clicked_template, neutrality_score=neutrality_score, image_filename=image_filename, image_score=image_score)

@app.route("/like", methods=["POST"])
def like():
    global neutrality_score
    post_id = request.form.get("post_id")
    liked_posts.add(post_id)
    neutrality_score = min(1.0, neutrality_score + 0.1)
    return jsonify({"success": True, "neutrality_score": neutrality_score})


@app.route("/dislike", methods=["POST"])
def dislike():
    global neutrality_score
    post_id = request.form.get("post_id")
    disliked_posts.add(post_id)
    neutrality_score = max(0.0, neutrality_score - 0.1)

    return jsonify({"success": True, "neutrality_score": neutrality_score})
# Run the Flask app
from werkzeug.serving import run_simple

if __name__ == "__main__":
    run_simple("localhost", 5001, app)

 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [16/Feb/2025 15:21:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2025 15:21:57] "GET /static/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Feb/2025 15:22:00] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2025 15:22:01] "GET /static/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Feb/2025 15:22:01] "GET /static/happyrabbit.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Feb/2025 15:22:01] "GET /static/desp_soldier.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Feb/2025 15:22:04] "POST /button-clicked/desp_soldier.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2025 15:22:04] "GET /static/desp_soldier.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Feb/2025 15:22:07] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2025 15:22:07] "GET /static/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Feb/2025 15:22:07] "GET /static/desp_soldier.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Feb/2025 15:22:07] "GET /static/happyrabbit.